In [205]:
import sys
sys.path.append('../common')
import utils
import importlib
importlib.reload(utils)
from utils import IOHandler as IO

import time
import random
import sys
import math

import numpy as np
import pynput.keyboard as kb

from collections import defaultdict

from multiprocessing import Pool
from functools import partial
from enum import Enum
from PIL import Image

io = utils.IOHandler(offset=(1081, 675), game_dims=(1200,900), verbose=True)

In [206]:
verbose = True

MAX_DIGITS = 6
MAX_CHANCES = 15

# TODO: use load_images_from_directory instead
level_masks = []
for i in range(31):
    img = Image.open('levels/mask_' + str(i+1) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    level_masks.append(arr)

# blank (10), none (11)
digit_masks = []
for i in range(12):
    img = Image.open('digits/mask_' + str(i) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    digit_masks.append(arr)

# green (0), red (1), blank (2)
ball_masks = []
for i in range(3):
    img = Image.open('balls/mask_' + str(i) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    ball_masks.append(arr)

time_masks = []
for i in range(10):
    img = Image.open('time/mask_' + str(i) + "_o.png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    time_masks.append(arr)

time_blank_mask = np.array(Image.open('time/mask_blank.png')).swapaxes(0, 1).astype(np.int32)[:,:,:3]

In [207]:
def debug(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)
        

class Screen(Enum):
    HOME = 'home'
    GAME_OVER = 'game_over'
    END_GAME = 'end_game'
    NEW_LEVEL = 'new_level'
    GREAT_JOB = 'great_job'
    ARCADE = 'arcade'
    MIDGAME = 'midgame'
    EASY_MODE = 'easy_mode'
    GREAT_PLAY = "great_play"

SCREEN_TYPES = {
    Screen.HOME: [1066, 684, [229, 99, 42]], # orange gumball
    Screen.GAME_OVER: [680, 380, [246, 196, 195]], # piggy bank
    Screen.END_GAME: [420, 860, [247, 207, 78]], # "end game" button, either ran out of guesses or time
    Screen.NEW_LEVEL: [850, 460, [245, 199, 69]], # "Start!", "Level: X", or congrats pop-up
    Screen.GREAT_JOB: [230, 664, [3, 14, 147]], # "great job!" pop-up
    Screen.ARCADE: [170, 852, [145, 61, 148]], # magenta arcade cabinet
    Screen.MIDGAME: [1095, 60, [92, 29, 196]], # purple music note
    Screen.EASY_MODE: [485, 44, [208, 126, 48]], # orange outline in level area
    Screen.GREAT_PLAY: [880, 600, [122, 26, 19]], # red gift box
}

io.set_screen_types(SCREEN_TYPES)

LEVEL_DIGITS = {
    'normal': [3, 3, 3, 4, 4, 4, 5, 5, 5, 6], 
    'easy': [3, 3, 3, 3, 4]
}
LEVEL_CHANCES = {
    'normal': [12,11,10]*3 + [11] + [11,10,9]*3 + [9] + [9,8,7]*3 + [8],
    'easy': [15]*5,
}
# Calculated using minimax (best worst eliminations) and expected information (entropy)
LEVEL_BEST_FIRST_GUESS = {
    'minimax': {
        False: ['012', '0123', '01234', '012345'], 
        True: ['012', '0123', '00123', '001123']
    },
    'entropy': {
        False: ['012', '0123', '01234', '001234'], 
        True: ['012', '0123', '01234', '001234']
    }
}
LEVEL_TIME = [120,140,160,190,210,230,260,280,300,330,90,110,130,160,180,200,230,250,270,300,60,80,100,130,150,170,200,220,240,270]

def get_level_digits(level, difficulty):
    return LEVEL_DIGITS[difficulty][(level-1) % 10]

def get_level_chances(level, difficulty):
    return LEVEL_CHANCES[difficulty][level-1]

def get_level_has_repeats(level):
    return level > 10

def get_level_time(level):
    return LEVEL_TIME[level-1]

def get_best_first_guess(num_digits, has_repeats, method='entropy'):
    return LEVEL_BEST_FIRST_GUESS[method][has_repeats][num_digits - 3]


def parse_level():
    if io.is_screen_type(Screen.EASY_MODE):
        level_img = io.capture_portion(615, 23, 45, 45)
    else:
        level_img = io.capture_portion(454, 23, 45, 45)
    curr_level = str(IO.get_best_mask(level_img, level_masks[:-1])[0] + 1)
    return curr_level

def parse_time():
    if io.is_screen_type(Screen.EASY_MODE):
        return -1
    time_img = io.capture_portion(751, 30, 68, 30)
    digit_minutes = IO.get_best_mask(time_img[:20, :30], time_masks)[0]
    digit_tens = IO.get_best_mask(time_img[28:28+20, :30], time_masks)[0]
    digit_ones = IO.get_best_mask(time_img[48:48+20, :30], time_masks)[0]
    return digit_minutes*60 + digit_tens*10 + digit_ones

def parse_digit(i, j):
    x0, y0 = (687,164)
    sx, sy = (32, 44)
    w, h = (30, 30)

    x = x0 + sx * j
    y = y0 + sy * i

    digit_img = io.capture_portion(x, y, w, h)

    idx = IO.get_best_mask(digit_img, digit_masks)[0]
    if idx == 10:
        digit = '-'
    elif idx == 11:
        digit = ''
    else:
        digit = str(idx)

    return digit

def parse_attempt(i):
    attempt = ''
    for j in range(MAX_DIGITS):
        attempt += parse_digit(i, j)
    return attempt

def parse_attempts():
    attempts = []
    for i in range (MAX_CHANCES):
        attempts.append(parse_attempt(i))
    return attempts

def capture_response_img(i):
    x0, y0 = (918, 165)
    w, h = (184, 13)
    sy = 44
    x = x0
    y = y0 + i*sy
    response_img = io.capture_portion(x, y, w, h)
    return response_img

def parse_response(i):
    w, h = (22, 13)
    sx = 30
    response_img = capture_response_img(i)
    response = [0, 0]
    for j in range (MAX_DIGITS):
        x = sx * j
        ball_img = response_img[x:x+w,:h, :]
        color = IO.get_best_mask(ball_img, ball_masks)[0]
        if color == 0:
            response[0] += 1
        elif color == 1:
            response[1] += 1
    return response

def parse_responses():
    return [parse_response(i) for i in range(MAX_CHANCES)]


# Returns the number of red & green gumballs
# TODO: possible to use LRU cache here or elsewhere? Or I/O too large?
def compare_codes(a, b):
    green = 0
    count1 = [0] * 10
    count2 = [0] * 10
    for dig1, dig2 in zip(a, b):
        if dig1 == dig2:
            green += 1
        else:
            count1[int(dig1)] += 1
            count2[int(dig2)] += 1
    red = sum(map(min, count1, count2))
    return int(green), int(red)

def unique_chars(s):
    return len(set(s)) == len(s)

def calc_worst_elims(word, possible):
    initial_count = len(possible)
    worst_elims = initial_count
    distribution = defaultdict(int)
    for p in possible:
        score = compare_codes(p, word)
        distribution[score] += 1
    for score, freq in distribution.items():
        num_elims = len(possible) - freq
        worst_elims = min(worst_elims, num_elims)
    return worst_elims

# TODO: Measure how much better this is than minimax ^, average guesses/points wise
def calc_expected_info(word, possible):
    initial_count = len(possible)
    expected_info = 0
    distribution = defaultdict(int)
    for p in possible:
        score = compare_codes(p, word)
        distribution[score] += 1
    for score, freq in distribution.items():
        prob = freq/initial_count
        info = math.log2(1/prob)
        expected_info += prob * info
    return expected_info

# TODO: move break/kill program to utils and make names/keys consistent in bots
def on_press(key):
    global break_program, listener
    if key == kb.Key.esc:
        print("Escape pressed to quit")
        listener.stop()
        break_program = True
    elif key == kb.Key.tab:
        print("Tab pressed to take screenshot")
        io.save_game_capture()

# TODO: move to utils?
def get_into_game(difficulty='normal'):
    global break_program
    while not break_program:
        screen_types = io.calc_screen_types()
        print(screen_types)
        
        if Screen.ARCADE in screen_types:
            io.click_mouse(970, 760, delays=[0.03, 0.03, 0]) # "PLAY GAME" button
        elif Screen.GREAT_PLAY in screen_types:
            io.click_mouse(590, 1000, delays=[0.03, 0.03, 0]) # "Collect" button
        elif Screen.GAME_OVER in screen_types:
            io.click_mouse(220, 703, delays=[0.03, 0.03, 0]) # Replay button
        elif Screen.HOME in screen_types:
            debug(f"Start game - difficulty {difficulty}")
            if difficulty == 'easy':
                io.click_mouse(450, 745, delays=[0.03, 0.03, 0]) # easy button
            else:
                io.click_mouse(690, 745, delays=[0.03, 0.03, 0]) # normal button
            io.click_mouse(185, 845, delays=[0.03, 0.03, 0]) # 1 player game button
            time.sleep(2)
            continue
        elif Screen.END_GAME in screen_types:
            io.click_mouse(320, 860, delays=[0.03, 0.03, 0]) # End Game button
        elif Screen.NEW_LEVEL in screen_types:
            pass
        elif Screen.GREAT_JOB in screen_types:
            time.sleep(5)
            continue
        elif Screen.MIDGAME in screen_types:
            return

        time.sleep(0.5)

def get_initial_level_info():
    level = int(parse_level())
    is_easy_mode = io.is_screen_type(Screen.EASY_MODE)
    difficulty = 'easy' if is_easy_mode else 'normal'
    num_digits = get_level_digits(level, difficulty)
    chances = get_level_chances(level, difficulty)
    has_repeats = get_level_has_repeats(level)
    
    return level, is_easy_mode, difficulty, num_digits, chances, has_repeats

def calc_initial_possible_unused(num_digits, has_repeats):
    possible = [str(i).zfill(num_digits) for i in range(10**(num_digits))]
    random.shuffle(possible)
    unused = possible
    if not has_repeats:
        possible = [code for code in possible if unique_chars(code)]
    return possible, unused

# TODO: Experiment with how long to spend searching
def calc_max_search_time():
    time_remaining = parse_time()
    if time_remaining > 0:
        max_search_time = time_remaining / 2 - 3
    else: # easy mode
        max_search_time = 60
    return max_search_time

# TODO: determine whether to search possible or unused
# TODO: stuff gets very messed up if I used unused. The expected info goes to zero...
def determine_search_set(possible, unused, num_digits):
    # return unused
    return possible
    if num_digits < 5:                        
        # unused = sorted(unused, key=lambda x: x in possible, reverse=True) # NOTE EXPENSIVE
        search_set = unused
    else:
        search_set = possible
    return search_set

def calc_best_next_guess(search_set, max_search_time, possible):
    debug(f"Searching ({max_search_time:.1f}s limit)...")
    results = []
    best_worst_elims = 0
    if len(search_set) == 0:
        debug("Something's rotten in Denmark, search_set is empty")
        return None

    search_start = time.time()
    for word in search_set:
        # result = calc_worst_elims(word, possible)
        result = calc_expected_info(word, possible)
        if result is None or break_program:
            debug("Broke program, cancel calc_best_next_guess")
            return None
        results.append(result)
        if time.time() - search_start >= max_search_time:
            debug("INFO: ran out of time to keep searching")
            break

    # TODO: fix Pool not working in ipynb
    # with Pool(8) as p:
    #     for result in p.imap(partial(calc_worst_elims, possible=possible, all_scores=all_scores), search_set, chunksize=1):
    #         results.append(result)
    #         if time.time() - search_start >= max_search_time:
    #             debug("INFO: ran out of time to keep searching")
    #             p.terminate()
    #             break
    #         print(f'Still searching... {result}')
    
    debug(f"searched {len(results)/len(search_set)*100:.1f}% ({len(results)}/{len(search_set)}) guesses in {time.time() - search_start:.2f}s")
    best_value = max(results)
    best_idx = results.index(best_value)
    suggestion = search_set[best_idx]
    # debug(f"min elims: {best_value}/{len(possible)}")
    debug(f"guess = {suggestion} has the best expected info = {best_value:.2f}")
    
    return suggestion

def submit_suggestion(suggestion, attempt_num, num_digits):
    # Type in suggestion
    MAX_DELETE_TIME = 3
    MAX_TYPE_TIME = 3
    MAX_TOTAL_TIME = 8
    attempt = parse_attempt(attempt_num)
    main_start = time.time()
    while not break_program and (len(attempt) != num_digits or '-' in attempt) and not io.is_screen_type(Screen.GREAT_JOB):   
        if time.time() - main_start > MAX_TOTAL_TIME:
            debug(f'Failed to submit suggestion. Spent >{MAX_DELETE_TIME} seconds trying in total. Assume game over.')
            return None
        time.sleep(0.1)
        correct_digits = 0
        for i, c in enumerate(attempt):
            if attempt[i] == suggestion[i]:
                correct_digits += 1
            else:
                break
        for i in range(len(attempt)-1, correct_digits-1, -1):
            start = time.time()
            while not break_program and attempt[i] != '-':
                if time.time() - start > MAX_DELETE_TIME:
                    debug(f'Failed to submit suggestion. Spent >{MAX_DELETE_TIME} seconds trying to backspace digits. Assume game over.')
                    return None
                time.sleep(0.1)
                debug("press backspace")
                io.click_key(kb.Key.backspace)
                attempt = parse_attempt(attempt_num)
        for i in range(correct_digits, len(attempt)):
            while not break_program and attempt[i] != suggestion[i]:  
                if time.time() - start > MAX_TYPE_TIME:
                    debug(f'Failed to submit suggestion. Spent >{MAX_TYPE_TIME} seconds trying to type digits. Assume game over.')
                    return None
                time.sleep(0.1)
                print(attempt, suggestion)
                debug("press {}".format(suggestion[i]))
                io.click_key(suggestion[i])
                attempt = parse_attempt(attempt_num)
                if len(attempt) <= i or len(suggestion) <= i:
                    debug('Trying to check an index for attempt/suggestion that is out of bounds (tsk tsk). Assume game over.')
                    return None
    if break_program:
        return None
    return attempt

def filter_possible_unused(possible, unused, attempt, score):
    possible = [p for p in possible if compare_codes(p, attempt) == score and p != attempt]
    if attempt in unused:
        unused.remove(attempt)
    return possible, unused

# Given a list of codes, will deduplicate them into ascending ABC paterns (convert to pattern, dedupe, keep ascending only)
# E.g. [000, 001, 002, 010, 011, 100, 101, 110, 111] -> [000, 001, 001, 010, 011, 011, 010, 001, 000] -> [000, 001, 010, 011] -> [000, 001, 011]
# Useful for finding good first guesses
# TODO: extend to be able to find second guesses based on digits in first guess. E.g. instead of [012, 779] --> [012, 334]
def dedupe_codes_to_patterns(codes):
    new_codes = set()
    for code in codes:
        i = 0
        char_map = defaultdict(int)
        new_code = ''
        for c in code:
            if c not in char_map:
                char_map[c] = i
                i += 1
            new_code += str(char_map[c])
        new_codes.add(new_code)
    return sorted([x for x in new_codes if ''.join(sorted(x)) == x])

# TODO: extend to make tree of decisions. E.g. what's best to guess after 001234 [1, 2]?
def calc_best_first_guess(num_digits, has_repeats):
    possible, unused = calc_initial_possible_unused(num_digits, has_repeats)
    # possible = sorted(possible)
    search_set = dedupe_codes_to_patterns(unused)
    suggestion = calc_best_next_guess(search_set, 600, possible)
    return suggestion

def wait_and_parse_response(num_digits, attempt_num):
    debug("response:\t", end='')
    MAX_RESPONSE_TIME = 5
    start = time.time()
    # Keep hammering '0' until the it pops up, this means the response has fully rolled in
    while not break_program and parse_attempt(attempt_num+1) == '-'*num_digits:
        if time.time() - start > MAX_RESPONSE_TIME:
            debug(f"WARNING: Waited too long (>{MAX_RESPONSE_TIME} seconds) for a response. Assume game over.")
            return None
        time.sleep(0.1)
        io.click_key("0")
    response = parse_response(attempt_num)
    debug(response)
    return response

In [208]:
break_program = False
listener = kb.Listener(on_press=on_press)
listener.start()

# TODO: faster suggestion finding!
# TODO: could make a seperate environment that doesn't have webkinz obstacles, direct interface for finding the best algo
# TODO: better time distribution for suggestion finding
# TODO: fix-up debug & print statements so more helpful & clear
# TODO: try to look further in the tree for best guesses.

io.click_mouse(0, 0, delays=[0.03, 0.03, 0])

while not break_program:
    desired_difficulty = 'normal'
    get_into_game(desired_difficulty)
    
    time.sleep(0.1)

    level, is_easy_mode, difficulty, num_digits, chances, has_repeats = get_initial_level_info()
    print(f"---LEVEL {level}: ({num_digits} digits, {'has' if has_repeats else 'no'} repeats, difficuly {difficulty})---")

    possible, unused = calc_initial_possible_unused(num_digits, has_repeats)

    attempt_num = 0

    while not break_program: # This loop completes a level
        time.sleep(0.1)
        debug("len(possible):\t", len(possible))
        
        attempt = parse_attempt(attempt_num)
        print (f"Attempt #{attempt_num}:\t{attempt}")
        if '-' in attempt: 
            if attempt_num == 0:
                suggestion = get_best_first_guess(num_digits, has_repeats)
            else:
                max_search_time = calc_max_search_time()
                search_set = determine_search_set(possible, unused, num_digits)
                suggestion = calc_best_next_guess(search_set, max_search_time, possible)
                if suggestion is None:
                    debug("Failed to calculate the next suggestion; assume game over.")
                    break

            print(f"Suggestion #{attempt_num}:\t{suggestion}")

            attempt = submit_suggestion(suggestion, attempt_num, num_digits)
            if attempt is None:
                debug(f"Submit {suggestion} was unsuccessful; assume game over.")
                break

        # If finished the level, wait for the next one
        if io.is_screen_type(Screen.GREAT_JOB):
            debug("Passed the level; wait for the next.")
            break
        # If failed the level, restart the game
        elif io.is_screen_type(Screen.END_GAME):
            debug("Failed the level; restart the game.")
            break
        
        response = wait_and_parse_response(num_digits, attempt_num)
        if response is None:
            debug("Unable to parse response; assume game over.")
            break
        num_greens, num_reds = int(response[0]), int(response[1])

        # If all green gumballs, we won; move to next level
        if num_greens == num_digits:
            debug(f"Level success! ({attempt_num + 1} attempts). Move to the next.")
            break
       
        possible, unused0 = filter_possible_unused(possible, unused, attempt, (num_greens, num_reds))
        
        attempt_num += 1

{<Screen.MIDGAME: 'midgame'>}
---LEVEL 1: (3 digits, no repeats, difficuly normal)---
len(possible):	 720
Attempt #0:	---
Suggestion #0:	012
--- 012
press 0
0-- 012
press 1
01- 012
press 2
response:	[0, 1]
len(possible):	 252
Attempt #1:	0--
Searching (43.0s limit)...
searched 100.0% (999/999) guesses in 0.87s
guess = 158 has the best expected info = 2.36
Suggestion #1:	158
press backspace
--- 158
press 1
1-- 158
press 5
15- 158
press 8
response:	[1, 0]
len(possible):	 45
Attempt #2:	0--
Searching (41.5s limit)...
searched 100.0% (998/998) guesses in 0.15s
guess = 283 has the best expected info = 2.47
Suggestion #2:	283
press backspace
--- 283
press 2
2-- 283
press 8
28- 283
press 3
response:	[0, 0]
len(possible):	 16
Attempt #3:	0--
Searching (40.0s limit)...
searched 100.0% (997/997) guesses in 0.05s
guess = 164 has the best expected info = 2.62
Suggestion #3:	164
press backspace
--- 164
press 1
1-- 164
press 6
16- 164
press 4
response:	[1, 0]
len(possible):	 2
Attempt #4:	0--
Search

KeyboardInterrupt: 